### Challenge 27: Recover the key from CBC with IV=Key

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)

<div class="alert alert-block alert-info">

Take your code from the CBC exercise and modify it so that it repurposes the key for CBC encryption as the IV.

Applications sometimes use the key as an IV on the auspices that both the sender and the receiver have to know the key already, and can save some space by using it as both a key and an IV.

Using the key as an IV is insecure; an attacker that can modify ciphertext in flight can get the receiver to decrypt a value that will reveal the key.

The CBC code from exercise 16 encrypts a URL string. Verify each byte of the plaintext for ASCII compliance (ie, look for high-ASCII values). Noncompliant messages should raise an exception or return an error that includes the decrypted plaintext (this happens all the time in real systems, for what it's worth).

Use your code to encrypt a message that is at least 3 blocks long:

```AES-CBC(P_1, P_2, P_3) -> C_1, C_2, C_3```

Modify the message (you are now the attacker):

```C_1, C_2, C_3 -> C_1, 0, C_1```

Decrypt the message (you are now the receiver) and raise the appropriate error if high-ASCII is found.

As the attacker, recovering the plaintext from the error, extract the key:

```P'_1 XOR P'_3```

</div>

The CBC bit-flipping attack was [Challenge 16](16.ipynb).  

In [1]:
import cryptopals as cp
from Crypto import Random

<div class="alert alert-block alert-info">
Generate a random AES key.
</div>

I just discovered the Crypto random library -- which has get_random_bytes and getrandbits.  Will use those from here out.

In [7]:
# ---------UNKNOWN PARAMETERS ------------
#unknown_key = Random.get_random_bytes(16)
unknown_key = b'DOGEATDOGEATDOGD'
unknown_IV = unknown_key
# ---------END UNKNOWN PARAMETERS---------

<div class="alert alert-block alert-info">

Use your code to encrypt a message that is at least 3 blocks long:

```AES-CBC(P_1, P_2, P_3) -> C_1, C_2, C_3```

</div>

In [8]:
P_1 = b'0123456789ABCDEF'
P_2 = b'0123456789ABCDEF'
P_3 = b'0123456789ABCDEF'

CT = cp.AESEncrypt(P_1+P_2+P_3, unknown_key, 'CBC', unknown_IV)

<div class="alert alert-block alert-info">
    
Modify the message (you are now the attacker):

```C_1, C_2, C_3 -> C_1, 0, C_1```
</div>

In [9]:
CT_malicious = CT[0:16] + b'\x00'*16 + CT[0:16] + CT[48:]

<div class="alert alert-block alert-info">

Decrypt the message (you are now the receiver) and raise the appropriate error if high-ASCII is found.

</div>

In [10]:
def Challenge27_decrypt_verify(CT, key, IV):
    
    PT = cp.AESDecrypt(CT, key, 'CBC', IV)    
    if any(int(PT) > 127):
        raise(ValueError(PT))

Call the function with our malicious ciphertext.  

When the exception returns the plaintext, scrape it off so we can use it to recover the key.

In [10]:
try:
    Challenge27_decrypt_verify(CT_malicious, unknown_key, unknown_IV)
except ValueError as Argument:
    
    PT = eval(Argument.args[0][40:])
    
print(PT)

b'0123456789ABCDEF\x1c\xfe\xc9od I\xe4\xb9;\tutr\xb8%t~uvuarx\x7f|\x00\x16\x07\x0b\x02\x02<\xdb\x0bf\xd8\xa6\xafx\x8d\xbf\x81]\x91\xfd9\x12'


<div class="alert alert-block alert-info">
    
As the attacker, recovering the plaintext from the error, extract the key:

```P'_1 XOR P'_3```

</div>

In [11]:
key_guess = cp.bitwise_xor(PT[0:16], PT[32:48])

if key_guess == unknown_key:
    print('You win')
    print(f'Extracted Key:  {key_guess}')
else:
    print('Try again')

You win
Extracted Key:  b'DOGEATDOGEATDOGD'


[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)